In [2]:
import os, shutil, glob
import PIL
from PIL import Image

#нужно скачать датасет https://data.mendeley.com/archiver/xnzhj3x8v4?version=2 вручную
#можно его же найти через поисковик по ключевым словам asphalt cracks dataset (скачивать version=2)
#фото асфальта без дефектов переложить в папку 0
#фото асфальта с дефектами переложить в папку 1
#я добавил asphalt.zip с этими картинками в проект

# функция do_5crop_and_flip делает 5 кропов углов и центра + для каждой из 5ти картинок выполняются опреации:
# horziontal flip
# vertical flip
# rotate_180 = horziontal flip + vertical flip
# за счет этого увеличиваем датасет в 20 раз

In [ ]:
# for unix
!unzip ../data/asphalt.zip
# move directories
!mv ./asphalt/1 ./1
!mv ./asphalt/0 ./0


In [3]:
def slash_index(img_path):
    for i in range(len(img_path)):
        if (img_path[-1-i] == '/'):
            return (len(img_path)-i-1)
    return 0



In [4]:


def get_coord_m10(w, h):
    minus = h//10
    return (0, 0, w, h-minus)


def get_coord_p10(w, h):
    minus = h//10
    return (0, minus, w, h)


def get_coord_c(w, h):
    return (0, 0, w, h)


def get_coord_cent(w, h):
    minus = (w-h)//2
    return (minus, 0, w-minus, h)


def get_coord_p50(w, h):
    minus = (w-h)//2
    return (minus-200, 0, w-minus-200, h)


def get_coord_m50(w, h):
    minus = (w-h)//2
    return (minus+200, 0, w-minus+200, h)


def get_coord_l(w, h):
    return (0, 0, w//2+1, h)


def get_coord_l_rect(w, h):
    minus = h//10
    return (0, 0, w//2+1, h-minus)


def save_flip_left_right(img1, outpath, final_name):
    img2 = img1.transpose(Image.FLIP_LEFT_RIGHT)
    img2 = img2.resize((224,224))
    img2.save(os.path.join(outpath, final_name))
    img2.close()


def save_flip_top_bottom(img1, outpath, final_name):
    img2 = img1.transpose(Image.FLIP_TOP_BOTTOM)
    img2 = img2.resize((224,224))
    img2.save(os.path.join(outpath, final_name))
    img2.close()


def save_rotate_180(img1, outpath, final_name):
    img2 = img1.transpose(Image.ROTATE_180)
    img2 = img2.resize((224,224))
    img2.save(os.path.join(outpath, final_name))
    img2.close()


def save_2flips_with_rotation(img1, outpath, n, name):
    save_rotate_180(img1, outpath, str(n+1500)+name)
    save_flip_top_bottom(img1, outpath, str(n+1000)+name)
    save_flip_left_right(img1, outpath, str(n+500)+name)


def save_ul2(img, name, n, outpath):
    wi, hi = img.size
    img1 = img.crop((0,0,wi-(wi//10),hi-(hi//10)))
    img2 = img1.resize((224,224))
    img2.save(os.path.join(outpath, str(n)+name))
    img2.close()
    save_2flips_with_rotation(img1, outpath, n, name)


def save_ur2(img, name, n, outpath):
    wi, hi = img.size
    img1 = img.crop(((wi//10),0,wi,hi-(hi//10)))
    img2 = img1.resize((224,224))
    img2.save(os.path.join(outpath, str(n)+name))
    img2.close()
    save_2flips_with_rotation(img1, outpath, n, name)


def save_mid2(img, name, n, outpath):
    wi, hi = img.size
    img1 = img.crop((wi//20,hi//20,wi-(wi//20),hi-(hi//20)))
    img2 = img1.resize((224,224))
    img2.save(os.path.join(outpath, str(n)+name))
    img2.close()
    save_2flips_with_rotation(img1, outpath, n, name)


def save_ll2(img, name, n, outpath):
    wi, hi = img.size
    img1 = img.crop((0,(hi//10),wi-(wi//10),hi))
    img2 = img1.resize((224,224))
    img2.save(os.path.join(outpath, str(n)+name))
    img2.close()
    save_2flips_with_rotation(img1, outpath, n, name)


def save_lr2(img, name, n, outpath):
    wi, hi = img.size
    img1 = img.crop((wi//10, hi//10, wi, hi))
    img2 = img1.resize((224,224))
    img2.save(os.path.join(outpath, str(n)+name))
    img2.close()
    save_2flips_with_rotation(img1, outpath, n, name)



In [5]:
def do_5crop_and_flip(imgpath, outpath, my_names):
    cnt = 0
    for name in my_names:
        img = Image.open(os.path.join(imgpath, name))
        wi, hi = img.size
        img1 = img
        save_ul2(img1, 'i_'+name, 0, outpath)
        save_ur2(img1, 'i_'+name, 1, outpath)
        save_mid2(img1, 'i_'+name, 2, outpath)
        save_ll2(img1, 'i_'+name, 3, outpath)
        save_lr2(img1, 'i_'+name, 4, outpath)
        img1.resize((224,224)).save(os.path.join(outpath, 'i'+name))
        img.close()
        cnt += 1
    print("done", cnt)



In [6]:
# функция do_5crop_and_flip делает 5 кропов углов и центра + для каждой из 5ти картинок выполняются опреации:
# horziontal flip
# vertical flip
# rotate_180 = horziontal flip + vertical flip
# за счет этого увеличиваем датасет в 20 раз

# создаем каталоги 00 11 для сконвертированных картинок
!mkdir "11"
!mkdir "00"

imgpath = './1'
outpath = './11'

my_names = [fn[slash_index(fn)+1:] for fn in glob.glob(os.path.join(imgpath,'*.jpg'))]
names = sorted(my_names)
print("sorted name 99: ", names[99])
do_5crop_and_flip(imgpath, outpath, names)

path_to_clean_asph = './0'
my_names = [fn[slash_index(fn)+1:] for fn in glob.glob(os.path.join(path_to_clean_asph,'*.jpg'))]
names = sorted(my_names)
print("name 90: ", names[90])
do_5crop_and_flip(path_to_clean_asph, './00', names)


sorted name 99:  188.jpg
done 204
name 90:  18.jpg
done 196
